# 11. Time Series

* fixed frequency: data points occur at regular intervals

ways to mark and refer to time series
* timestamps
* Fixed periods, e.g., Jan 2007, or 2010
* Intervals
    * start and end timestamp (? no interval)
    * periods are special cases of intervals (?)
* Experiment or elapsed time
    * start time, relative time
    
Notes
* timedelta indexes of pandas is not covered in the book

## 11.1 Date and Time Data Types and Tools

In [1]:
from datetime import datetime

now = datetime.now()

now

datetime.datetime(2021, 4, 27, 12, 33, 36, 92270)

In [2]:
now.year, now.month, now.day

(2021, 4, 27)

In [3]:
from datetime import timedelta

start = datetime(2011, 1, 7)
start + timedelta(12) # day

datetime.datetime(2011, 1, 19, 0, 0)

In [4]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

### Converting Between String and Datetime

In [5]:
stamp = datetime(2021, 3, 27)
str(stamp)

'2021-03-27 00:00:00'

In [6]:
# string-format-time
stamp.strftime('%Y-%m-%d')

'2021-03-27'

#### Datetime format specification (ISO C89 compatible)

|Type|Description|
|----|:----------------|
|%Y|Four-digit year|
|%y|Two-digit year|
|%m|Two-digit month [01, 12]|
|%d|Two-digit day [01, 31]|
|%H|Hour (24-hour clock) [00, 23]|
|%I|Hour (12-hour clock) [01, 12]|
|%M|Two-digit minute [00, 59]|
|%S|Second [00, 61] (seconds 60, 61 account for leap seconds)|
|%w|Weekday as integer [0 (Sunday), 6]|
|%U|Week number of the year [00, 53]; Sunday is considered the first day of the week, and days before the first Sunday of the year are “week 0”|
|%W|Week number of the year [00, 53]; Monday is considered the first day of the week, and days before the first Monday of the year are “week 0”|
|%z|UTC time zone offset as +HHMM or -HHMM; empty if time zone naive|
|%F|Shortcut for %Y-%m-%d (e.g., 2012-4-18)|
|%D|Shortcut for %m/%d/%y (e.g., 04/18/12)|

#### Locale-specific date formatting

|Type|Description|
|:---|:---|
|%a|Abbreviated weekday name|
|%A|Full weekday name|
|%b|Abbreviated month name|
|%B|Full month name|
|%c|Full date and time (e.g., ‘Tue 01 May 2012 04:20:57 PM’)|
|%p|Locale equivalent of AM or PM|
|%x|Locale-appropriate formatted date (e.g., in the United States, May 1, 2012 yields ’05/01/2012’)|
|%X|Locale-appropriate time (e.g., ’04:24:12 PM’)|

In [7]:
value = '2011-01-03'
# string-parse-time
datetime.strptime(value, '%Y-%M-%d')

datetime.datetime(2011, 1, 3, 0, 1)

In [8]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [9]:
# 3rd party dateutil package that is installed with pandas
from dateutil.parser import parse

parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [10]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [11]:
parse('6/12/2022', dayfirst=True)

datetime.datetime(2022, 12, 6, 0, 0)

In [12]:
import pandas as pd

datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

In [13]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [14]:
idx[2] # Not a Time

NaT

In [15]:
pd.isnull(idx)

array([False, False,  True])

## 11.2 Time Series Basics

In [16]:
import numpy as np
from datetime import datetime

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(range(6), index=dates)

ts

2011-01-02    0
2011-01-05    1
2011-01-07    2
2011-01-08    3
2011-01-10    4
2011-01-12    5
dtype: int64

In [17]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [18]:
ts + ts[::2]

2011-01-02    0.0
2011-01-05    NaN
2011-01-07    4.0
2011-01-08    NaN
2011-01-10    8.0
2011-01-12    NaN
dtype: float64

In [19]:
ts.index.dtype

dtype('<M8[ns]')

In [20]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

### Indexing, Selection, Subsetting

In [21]:
ts[stamp]

0

In [22]:
# US date notation
ts['1/10/2011']

4

In [78]:
ts

2012-03-09 09:30:00    0
2012-03-10 09:30:00    1
2012-03-11 09:30:00    2
2012-03-12 09:30:00    3
2012-03-13 09:30:00    4
2012-03-14 09:30:00    5
Freq: D, dtype: int64

In [81]:
# filtering by day
ts[ts.index.strftime('%d') == '09']

2012-03-09 09:30:00    0
Freq: D, dtype: int64

In [23]:
try:
    ts['10/1/2011']
except:
    print("The Europe notation doesn't work.")

The Europe notation doesn't work.


In [24]:
# This works though
ts['2011/1/10']

4

In [25]:
ts['20110110']

4

In [26]:
longer_ts = pd.Series(np.random.randn(1000),
                     index=pd.date_range('1/1/2021', periods=1000))
longer_ts

2021-01-01   -0.248222
2021-01-02   -0.419868
2021-01-03    0.050198
2021-01-04   -1.341453
2021-01-05    0.719651
                ...   
2023-09-23    0.320258
2023-09-24   -0.586600
2023-09-25    0.853098
2023-09-26    0.430836
2023-09-27   -1.446858
Freq: D, Length: 1000, dtype: float64

In [27]:
longer_ts['2021-02']

2021-02-01    0.166493
2021-02-02    0.436174
2021-02-03    0.682410
2021-02-04    1.404001
2021-02-05   -0.364566
2021-02-06    1.077395
2021-02-07    1.954459
2021-02-08    0.028510
2021-02-09   -0.547334
2021-02-10   -0.364851
2021-02-11    0.491607
2021-02-12   -0.737334
2021-02-13   -0.935046
2021-02-14    1.271947
2021-02-15   -0.931945
2021-02-16   -0.154753
2021-02-17    0.776267
2021-02-18    2.164868
2021-02-19    1.213700
2021-02-20   -0.251251
2021-02-21   -0.834747
2021-02-22    0.230838
2021-02-23    0.713707
2021-02-24   -0.770621
2021-02-25    0.475065
2021-02-26   -0.687544
2021-02-27   -2.222414
2021-02-28    0.647998
Freq: D, dtype: float64

In [28]:
# slicing
ts[datetime(2011, 1, 7):]

2011-01-07    2
2011-01-08    3
2011-01-10    4
2011-01-12    5
dtype: int64

In [29]:
ts['20110106':'20110112']

2011-01-07    2
2011-01-08    3
2011-01-10    4
2011-01-12    5
dtype: int64

In [30]:
ts.truncate(after='20110109')

2011-01-02    0
2011-01-05    1
2011-01-07    2
2011-01-08    3
dtype: int64

### Time Series with Dulicate Indices

In [31]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])

dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [32]:
dup_ts.index.is_unique

False

In [33]:
dup_ts['20000101']

0

In [34]:
dup_ts['20000102']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [35]:
grouped = dup_ts.groupby(level=0)
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [36]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3 Date Ranges, Frequencies, and Shifting

In [37]:
ts

2011-01-02    0
2011-01-05    1
2011-01-07    2
2011-01-08    3
2011-01-10    4
2011-01-12    5
dtype: int64

In [38]:
resampler = ts.resample('D')
resampler

### Generating Date Ranges

In [39]:
index = pd.date_range('2012-04-01', '2012-05-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01'],
              dtype='datetime64[ns]', freq='D')

In [40]:
pd.date_range(start='2021-04-01', periods=20)

DatetimeIndex(['2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04',
               '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
               '2021-04-09', '2021-04-10', '2021-04-11', '2021-04-12',
               '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16',
               '2021-04-17', '2021-04-18', '2021-04-19', '2021-04-20'],
              dtype='datetime64[ns]', freq='D')

In [41]:
pd.date_range(end='2021-04-01', periods=20)

DatetimeIndex(['2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16',
               '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20',
               '2021-03-21', '2021-03-22', '2021-03-23', '2021-03-24',
               '2021-03-25', '2021-03-26', '2021-03-27', '2021-03-28',
               '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01'],
              dtype='datetime64[ns]', freq='D')

In [42]:
# BM: BusinessMonthEnd / Last business day (weekday) of month
pd.date_range('2000-01-01', '2000-12-01', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [43]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [44]:
# nomalize to midnight
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### Frequencies and Date Offsets

Frequencies = base frequency + multiplier

In [45]:
from pandas.tseries.offsets import Hour, Minute

hour = Hour()
hour

<Hour>

In [46]:
four_hours = Hour(4)
four_hours

<4 * Hours>

In [47]:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [48]:
Hour(2) + Minute(30)

<150 * Minutes>

In [49]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

Anchored offsets: not evenly spaced frequencies like 'M' and 'BM'.

In [50]:
# Week of Month Dates
# This shows third Friday of each month.
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### Shifting (Leading and Lagging) Data

In [51]:
ts = pd.Series(np.arange(4),
               index=pd.date_range('1/1/2000', periods=4, freq='M'))

ts

2000-01-31    0
2000-02-29    1
2000-03-31    2
2000-04-30    3
Freq: M, dtype: int64

In [52]:
ts.shift(2)

2000-01-31    NaN
2000-02-29    NaN
2000-03-31    0.0
2000-04-30    1.0
Freq: M, dtype: float64

In [53]:
ts.shift(-2)

2000-01-31    2.0
2000-02-29    3.0
2000-03-31    NaN
2000-04-30    NaN
Freq: M, dtype: float64

In [54]:
ts / ts.shift(1) - 1

2000-01-31    NaN
2000-02-29    inf
2000-03-31    1.0
2000-04-30    0.5
Freq: M, dtype: float64

In [55]:
# advancing the timestamp
ts.shift(2, freq='M')

2000-03-31    0
2000-04-30    1
2000-05-31    2
2000-06-30    3
Freq: M, dtype: int64

In [56]:
# T: minutes
ts.shift(1, freq='90T')

2000-01-31 01:30:00    0
2000-02-29 01:30:00    1
2000-03-31 01:30:00    2
2000-04-30 01:30:00    3
dtype: int64

#### Shifting dates with offsets

In [57]:
from pandas.tseries.offsets import Day, MonthEnd

now = datetime(2011, 11, 17)
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [58]:
# rolling forward to the next anchored date
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [59]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

In [60]:
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [61]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

In [62]:
ts = pd.Series(np.arange(10),
               index=pd.date_range('1/15/2000', periods=10, freq='4d'))

ts

2000-01-15    0
2000-01-19    1
2000-01-23    2
2000-01-27    3
2000-01-31    4
2000-02-04    5
2000-02-08    6
2000-02-12    7
2000-02-16    8
2000-02-20    9
Freq: 4D, dtype: int64

In [63]:
ts.groupby(offset.rollforward).mean()

2000-01-31    2
2000-02-29    7
dtype: int64

In [64]:
# in fact, this is easier and faster
ts.resample('M').mean()

2000-01-31    2
2000-02-29    7
Freq: M, dtype: int64

## 11.4 Time Zone Handling

In [65]:
import pytz

pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [66]:
pytz.common_timezones[:5]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara']

In [67]:
len(pytz.common_timezones)

440

In [68]:
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [69]:
pytz.timezone('Europe/Berlin')

<DstTzInfo 'Europe/Berlin' LMT+0:53:00 STD>

In [70]:
{tz.split('/')[0] for tz in pytz.common_timezones}

{'Africa',
 'America',
 'Antarctica',
 'Arctic',
 'Asia',
 'Atlantic',
 'Australia',
 'Canada',
 'Europe',
 'GMT',
 'Indian',
 'Pacific',
 'US',
 'UTC'}

In [71]:
[tz for tz in pytz.common_timezones if 'US' in tz]

['US/Alaska',
 'US/Arizona',
 'US/Central',
 'US/Eastern',
 'US/Hawaii',
 'US/Mountain',
 'US/Pacific']

### Time Zone Localization and Conversion

In [72]:
# By default, time series in pandas are time zone naive. 
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = pd.Series(np.arange(len(rng)), index=rng)
ts

2012-03-09 09:30:00    0
2012-03-10 09:30:00    1
2012-03-11 09:30:00    2
2012-03-12 09:30:00    3
2012-03-13 09:30:00    4
2012-03-14 09:30:00    5
Freq: D, dtype: int64

In [73]:
print(ts.index.tz)

None


In [74]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [75]:
# naive to localized
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00    0
2012-03-10 09:30:00+00:00    1
2012-03-11 09:30:00+00:00    2
2012-03-12 09:30:00+00:00    3
2012-03-13 09:30:00+00:00    4
2012-03-14 09:30:00+00:00    5
Freq: D, dtype: int64

In [76]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [77]:
ts_utc.tz_convert('Europe/Berlin')

2012-03-09 10:30:00+01:00    0
2012-03-10 10:30:00+01:00    1
2012-03-11 10:30:00+01:00    2
2012-03-12 10:30:00+01:00    3
2012-03-13 10:30:00+01:00    4
2012-03-14 10:30:00+01:00    5
Freq: D, dtype: int64

**Caution**

Localizing naive timestamps also checks for ambiguous or non-existent times around daylight saving time transitions.

???

### Operations with Time Zone−Aware Timestamp Objects